# 📊 Desafio Técnico – Gabriel Alves

##  Descrição do Projeto:
Este notebook foi desenvolvido como parte do processo seletivo para a vaga de Cientista de Dados, com o objetivo de construir um chatbot de análise de dados com LLM.

In [101]:
# Instalação de bibliotecas

!pip install boto3 pandas openai gradio matplotlib --quiet

In [102]:
# Importação de bibliotecas

import boto3
import pandas as pd
from openai import OpenAI
import sqlite3
import gzip
from io import BytesIO, StringIO
import matplotlib.pyplot as plt
import gradio as gr
import tempfile

In [103]:
# Credenciais de acesso e configurações

aws_access_key = 'Insira sua credencial aqui'
aws_secret_key = 'Insira sua credencial aqui'
client = OpenAI(api_key='Insira sua credencial aqui')

bucket_name = 'bucket-desafio-ds'

s3 = boto3.client(
    's3',
    region_name='us-east-2',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

In [104]:
# Função para validar a existência do arquivo no bucket da S3

def detect_file():
    response = s3.list_objects_v2(Bucket=bucket_name)
    arquivos = [obj['Key'] for obj in response.get('Contents', [])]
    if 'train.csv.gz' in arquivos:
        return 'train.csv.gz', 'GZIP'
    elif 'train.csv' in arquivos:
        return 'train.csv', 'NONE'
    else:
        raise FileNotFoundError("Arquivo esperado não encontrado no bucket.")


In [105]:
#Função para gerar a query SQL via GPT

def generate_sql_query(consulta):
    prompt = f"""
Você é um assistente especializado em converter perguntas em queries SQL para arquivos CSV.
A base possui as colunas: REF_DATE, TARGET, VAR2, IDADE, VAR4, VAR5, VAR8.
Gere uma query SQL para responder à pergunta: "{consulta}"
Considere que:
- TARGET: 0 é adimplente, 1 é inadimplente
- VAR2 é sexo M para masculino ou F para feminino, VAR4 FLAG DE ÓBITO, VAR5 é UF, VAR8 é classe social estimada
- A tabela se chama "s3object"
- Use aspas duplas para nomes de colunas.
"""
    resposta = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=300
    )

    conteudo = resposta.choices[0].message.content
    conteudo = conteudo.replace("```sql", "").replace("```", "").strip()
    return conteudo

In [106]:
#Função de leitura dos dados via SQL

def local_query(sql_query, object_key, compress_type):
    response = s3.get_object(Bucket=bucket_name, Key=object_key)
    content = response['Body'].read()
    if compress_type == 'GZIP':
        content = gzip.decompress(content)
    csv_str = content.decode('utf-8')
    df = pd.read_csv(StringIO(csv_str))

    conn = sqlite3.connect(":memory:")
    df.to_sql("s3object", conn, index=False, if_exists='replace')
    resultado_df = pd.read_sql_query(sql_query, conn)
    conn.close()
    return resultado_df

In [107]:
#Função de geração de insights via GPT

def generate_insights(df, consulta):

    # obs: A quantidade de linhas do recorte do dataframe a serem analisadas para geração de insights pode ser ajustado no parâmetro 'df.head'

    if df.empty:
        return "Consulta não retornou dados para gerar insights."

    descricao = f"""
Você é um analista de dados. Aqui estão os dados resultantes da seguinte consulta: "{consulta}".
Gere 2 a 3 insights automáticos baseados no DataFrame a seguir:

{df.head(1000).to_string(index=False)}

Seja direto, técnico e objetivo.
"""
    resposta = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": descricao}],
        temperature=0.3,
        max_tokens=300
    )
    return resposta.choices[0].message.content.strip()

In [108]:
# Função para gerar o gráfico via matplotlib

def generate_plot(df, consulta):

    # Validação mínima para gerar gráfico
    if df.empty:
        return None

    # Detectar colunas
    cols = df.columns.tolist()
    if len(cols) < 2:
        return None

    # Detectar colunas para eixo x e y
    # x: primeira coluna não numérica ou de data, y: primeira numérica
    x_col = None
    y_col = None
    for c in cols:
        if pd.api.types.is_numeric_dtype(df[c]):
            if y_col is None:
                y_col = c
        else:
            if x_col is None:
                x_col = c

    if x_col is None or y_col is None:
        # fallback: plot só a primeira coluna numérica como histograma
        plt.figure(figsize=(8,5))
        plt.hist(df[cols[0]])
        plt.title("Histograma de " + cols[0])
        plt.tight_layout()
        return plt

    # Gráfico de barras agrupando valores únicos do x_col
    plt.figure(figsize=(8,5))
    try:
        grouped = df.groupby(x_col)[y_col].mean().sort_values(ascending=False)
        grouped.plot(kind='bar')
        plt.title(f"Média de {y_col} por {x_col}")
        plt.xlabel(x_col)
        plt.ylabel(f"Média de {y_col}")
        plt.xticks(rotation=45)
        plt.tight_layout()
        return plt
    except Exception as e:
        return None

In [109]:
# Função de salvar o .csv

def save_csv(df):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
    df.to_csv(temp_file.name, index=False)
    return temp_file.name

In [110]:
# Função principal

def execute_pipeline(consulta):
    try:
        # Detecta arquivo no S3
        object_key, compress_type = detect_file()

        # Gera SQL com OpenAI
        sql_query = generate_sql_query(consulta)

        # Consulta e retorna dataframe
        df = local_query(sql_query, object_key, compress_type)

        if df.empty:
            return ("", "Consulta retornou 0 linhas.", "Sem dados para insights.", None, None)

        # Formação de datas
        if 'REF_DATE' in df.columns:
            df['REF_DATE'] = pd.to_datetime(df['REF_DATE'], errors='coerce').dt.strftime('%d/%m/%Y')

        # Gerar insights
        insights = generate_insights(df, consulta)

        # Gerar gráfico matplotlib
        plt_fig = generate_plot(df, consulta)

        # Salvar CSV para download
        arquivo_path = save_csv(df)

        # Preparar saída texto para tabela e SQL
        tabela_preview = df.head(50).to_string(index=False)

        return (sql_query, tabela_preview, insights, plt_fig, arquivo_path)

    except Exception as e:
        erro_msg = f"O código teve o seguinte erro: {str(e)}"
        return ("", erro_msg, "", None, None)

In [111]:
# Interface Gradio

with gr.Blocks(theme=gr.themes.Soft(primary_hue="violet", secondary_hue="gray")) as demo:
    gr.Markdown("## 📊 Desafio Data Science - Gabriel Alves")
    consulta = gr.Textbox(label="🔍 Insira seu prompt para consulta (ex: 'inadimplência por UF')")
    executar = gr.Button("Executar")

    sql_output = gr.Textbox(label="🧠 SQL Gerada", interactive=False)
    tabela = gr.Textbox(label="📄 Resultado dos Dados", lines=10, interactive=False)
    insights_box = gr.Textbox(label="💡 Insights", lines=6, interactive=False)
    grafico_output = gr.Plot(label="📈 Gráfico Gerado")
    arquivo_csv = gr.File(label="📁 Baixar CSV", file_types=[".csv"])

    executar.click(
        fn=execute_pipeline,
        inputs=consulta,
        outputs=[sql_output, tabela, insights_box, grafico_output, arquivo_csv]
    )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac538abb778bc42c19.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
